In [5]:
import gradio as gr
from PIL import Image
from dotenv import load_dotenv
import urllib
from openai import OpenAI 
import google.generativeai as genai
import numpy as np 

load_dotenv()

client = OpenAI()
genai.configure()

In [15]:
# 사용자 및 시스템 메시지 기록
user_messages = []
system_messages = []
global user_info

# 글로벌 변수 초기화
relation = "배우자"
closeness = "10"

# 관계와 가까움 정도 설정하는 함수
def setting_gemini(input_relation, input_closeness):
    global relation, closeness
    relation = input_relation
    closeness = input_closeness

def create_letter(input_image, input_title, input_text):
    try:
        if isinstance(input_image, np.ndarray):
            img = Image.fromarray(input_image)
        else:
            raise ValueError("입력된 이미지가 올바르지 않습니다.")
    except Exception as e:
        return gr.update(value=""), gr.update(value=f"이미지 처리 오류: {e}")

    title = input_title
    text = input_text

    model = genai.GenerativeModel('gemini-1.0-pro-vision-latest')
    prompt = f"""
    당신의 역할
    역할: 편지를 받는 대상이 되어서 사용자에게 답장을 써라
    편지 내용: 죽은 대상에게 그리운 마음을 담아 편지를 작성함
    답장 내용: 편지 내용을 기반으로 답장을 작성해야하며 죽은 입장에서 편지에 대한 답장을 작성함
    편지 제목: {input_title}
    당신에 대한 정보: {user_info}

    {input_text}에 대한 답장을 작성하라.
    당신에게 편지를 보내는 사람은{user_info["userName"]}이다.
    당신의 이름은 {user_info['name']}이고, 당신은 {relation}이며, {closeness} 정도로 가까웠다.
    답장의 문체는 {user_info['tone']}을 참고해서 작성하여라.
    편지의 시작은 "사랑하는 {user_info["userName"]}이에게"로 시작하며 편지의 끝은 "사랑하는 {user_info['name']}이가"를 붙여야 한다.
    답장의 예시는 다음과 같다.
"""

    try:
        response = model.generate_content([prompt, img, text])
        reply_content = response.candidates[0].content.parts[0].text
    except Exception as e:
        return gr.update(value=""), gr.update(value=f"답장 생성 오류: {e}")

    return gr.update(value=reply_content)


# 페이지 전환 함수
def switch_tab(tab_index):
    if tab_index == 0:
        return gr.update(visible=True), gr.update(visible=False)
    elif tab_index == 1:
        return gr.update(visible=False), gr.update(visible=True)
    

# 설정 값 저장 함수
def save_settings(image, name, message):
    global current_image, current_name, current_message
    current_image = image
    current_name = name
    current_message = message

    return gr.update(value=current_image), gr.update(value=current_name), gr.update(value=current_message)
    

# 수집된 정보 저장
user_info = {
    "entity_type": None,
    "name": None,
    "text": None,
    "age": None,
    "tone": None,
    "userName": None
}

def save_name_settings(name):
    global user_info
    name = user_info['name']

    return gr.update(value=name)

def save_text_settings(text):
    global user_info
    text = user_info['text']

    return gr.update(value=text)

# 현재 질문 상태 저장
current_question = None

# 수집되지 않은 정보 확인
def get_missing_info():
    return [key for key, value in user_info.items() if value is None]

# 사용자 입력 처리 함수
def process(prompt, history):
    global user_messages, system_messages, user_info, current_question

    user_messages.append({"role": "user", "content": prompt})

    # 현재 질문 상태에 따라 적절히 정보 저장
    if current_question == "entity_type":
        if "사람" in prompt:
            user_info["entity_type"] = "사람"
        else:
            user_info["entity_type"] = "동물"
    elif current_question == "name":
        user_info["name"] = prompt
    elif current_question == "text":
        user_info["text"] = prompt
    elif current_question == "age":
        user_info["age"] = ''.join(filter(str.isdigit, prompt))
    elif current_question == "tone":
        user_info["tone"] = prompt
    elif current_question == "userName":
        user_info["userName"] = prompt


    # 다음에 물어볼 질문을 결정
    missing_info = get_missing_info()
    if missing_info:
        if "entity_type" in missing_info:
            current_question = "entity_type"
            bot_response = "안녕하세요! 저희는 추모 대상의 정보를 수집하는 챗봇입니다. 채팅을 통해 얻은 정보는 추모 대상에 대한 소중한 추억을 만들어갈 예정이니 자세한 정보를 알려주시면 정말 감사할 것 같아요. \n 추모 대상은 사람인가요, 동물인가요?"
        elif "name" in missing_info:
            current_question = "name"
            bot_response = "그렇다면 추모 대상의 아름다운 이름을 알 수 있을가요? 이곳에 기록되어 추모의 의미를 더 특별하게 만들어 드릴게요."
        elif "text" in missing_info:
            current_question = "text"
            bot_response = "그를 한문장으로 표현한다면 어떻게 말할 수 있을까요?"
        elif "age" in missing_info:
            current_question = "age"
            bot_response = "이제는 추모 대상의 나이를 알려주세요. 그의 생애를 존경하며 기억할 수 있도록 도와드릴게요."
        elif "tone" in missing_info:
            current_question = "tone"
            bot_response = "추모 대상과의 소중한 대화를 상기시켜주세요. 그의 말투를 알려주시면 추모의 의미가 더 깊어질 거예요."
        elif "userName" in missing_info:
            current_question = "userName"
            bot_response = "마지막으로 당신의 이름을 알려주세요"
    else:
        current_question = None
        bot_response = "모든 정보를 수집하였습니다. 저에게 주신 정보들로 추모 대상에 대한 소중한 기억을 만들기 위해 노력할게요. 귀중한 시간을 내주셔서 감사합니다. 하단의 메인으로 넘어가기 버튼을 눌러주세요."

    system_messages.append({"role": "system", "content": bot_response})

    # 확인용 출력값(서비스에서는 지우기)
    return f"BOT: {bot_response}"



class ConversationLogger:
    def __init__(self):
        self.user_messages = []
        self.system_messages = []

    def log_user_message(self, message):
        self.user_messages.append(message)

    def log_system_message(self, message):
        self.system_messages.append(message)

logger = ConversationLogger()

chatbot_role = [ #챗봇 역할
    """
	역할: 비애상담가 또는 애도상담가
	상담 대상: 사랑하는 이를 잃은 사람들 (고인 또는 반려동물)
	목표: 상담을 통해 사용자가 일상생활을 되찾도록 도와주는 것
	사용자의 상황과 증상 파악
	말투: 상대방의 감정을 잘 이해하고 공감하는 대화 스타일
"""
]

examples_of_progress = [ #상담 진행 방식
    """
1. 애도 상담

자살로 인한 사망의 충격과 슬픔을 겪는 유족
유족의 심리적 고통은 매우 강렬하고 복합적
우울증 포함 부정적 감정 지속
사회적 편견으로 인한 고립
애도 상담의 유의미함

질문: "최근에 겪으신 일에 대해 좀 더 이야기해주실 수 있을까요?"
공감: "정말 힘드셨겠어요. 그 슬픔과 고통이 느껴져요."
다음 질문: "그분과의 소중한 기억이 있으신가요? 기억을 떠올리는 것만으로도 도움이 될 수 있어요."


2. 사별 받아들이기

사랑하는 가족의 갑작스러운 죽음의 충격과 다양한 감정
사별 이후의 생활 영역 변화
긴급 문제 해결 필요
고인에 대한 이야기 연습 필요

질문: "고인과의 관계가 정말 특별하셨나 봐요. 어떻게 지내셨는지 들려주시겠어요?"
공감: "그분을 잃은 것이 얼마나 큰 충격이었을지 이해가 돼요."
다음 질문: "사별 후에 생활에서 어떤 변화가 있으셨나요? 어떤 부분이 가장 힘드셨는지 이야기해주시면 좋겠어요."


3. 애도 과정 겪어내기

무감각, 갈망, 그리움, 절망감, 분노, 재조직화 등의 과정
애도 과정에서의 심리‧정서‧신체의 어려움
급성 스트레스 장애와 관련된 증상


질문: "최근에 어떤 감정의 변화가 있으셨나요? 슬픔이나 분노를 느끼셨나요?"
공감: "그런 감정을 느끼는 것은 자연스러운 일이에요. 혼자가 아니에요."
다음 질문: "어떻게 그런 감정들을 다루고 계신가요? 우리가 함께 더 나은 방법을 찾아볼 수 있어요."
"""
]

required_questions = [ # 상담 질문 예시
    "최근에 겪으신 일에 대해 좀 더 이야기해주실 수 있을까요?",
    "고인과의 관계가 어떠했는지, 함께한 소중한 기억이 있으신가요?",
    "사별 이후에 생활에서 어떤 변화가 있으셨나요? 어떤 부분이 가장 힘드셨나요?",
    "최근에 어떤 감정의 변화가 있으셨나요? 슬픔이나 분노를 느끼셨나요?",
    "그런 감정들을 어떻게 다루고 계신가요?",
    "고인과의 소중한 기억이 있으신가요? 기억을 떠올리는 것만으로도 도움이 될 수 있어요.",
    "이별 후 어떤 신체적 증상을 겪고 계신가요? (예: 잦은 울음, 불면, 식욕 변동 등)",
    "이별 후 어떤 정서적 변화를 겪고 계신가요? (예: 슬픔, 우울, 불안 등)",
    "이별 후 어떤 인지적 변화를 겪고 계신가요? (예: 주의 집중 어려움, 죽음에 대한 생각 몰두 등)",
    "일상생활로 돌아가기 위해 어떤 지원이 필요하신가요?"
]



system = f"""
당신의 역할: {chatbot_role}
상담 방법
사용자에게 직접 증상을 묻기보다는 상황을 헤아리며 유연하게 질문하기
사용자 질문 하나 다음에 공감을 표시하고 증상을 해결할 수 있는 질문 진행
상대방의 감정을 이해하고 공감하는 대화 스타일을 유지해주시오


사랑하는 이와 이별 이후 어떠한 증상을 겪고 있는 지에 대한 질문 필요

신체적/행동적 변화: 잦은 울음, 안절부절, 악몽, 불면, 식욕 및 수면 변동, 통증, 멍함
정서적 변화: 슬픔, 우울, 희망 없음, 울화, 죄책감, 급격한 기분 변화, 불안, 무감각
인지적 변화: 관계 상실에 대한 회피 및 부정, 냉소주의적 태도, 회의적인 생각, 주의 집중 어려움,
죽음에 대한 생각 몰두, 상담 과정


상담 진행 방식: {examples_of_progress} 다음과 같은 단계로 상담 진행

질문 목록: {required_questions}

"""

messages = [ {"role": "system", "content": system}]

# 이전 사용자 대화 기록을 저장하기 위한 리스트
user_messages = []
system_messages = []

current_question_index = 0



def process_chat(prompt, history):
    global current_question_index
    
    logger.log_user_message(prompt)
    
    messages = [{"role": "system", "content": system}] + [{"role": "user", "content": message} for message in logger.user_messages + [prompt]]
    
    try:
         completion = client.chat.completions.create(
					model="gpt-3.5-turbo",
					messages=[
						{"role": "system", "content": system},
						{"role": "user", "content": prompt}]
				)
         return "당신의 심리 상담사:" + completion.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        response = "죄송합니다. 현재 ChatGPT API에 문제가 있는 것 같습니다. 잠시 후에 다시 시도해 주세요."


    logger.log_system_message(response)


# Gradio 인터페이스 생성
with gr.Blocks() as demo:
    with gr.Column(visible=True) as start_page:
        chatbot=gr.ChatInterface(process, title="그리운 대상을 떠올려보세요", description="준비가 됐다면 간단하게 제게 인사해주세요",fill_height=False)
        btn_settings_1 = gr.Button("메인으로 넘어가기")

    with gr.Blocks() as main_interface:
        with gr.Row():
            # 메인 화면
            with gr.Column(visible=False) as main_page:
                gr.Markdown("# 추모 공간")
                image_display = gr.Image(type="pil", label="사랑하는 나의", height=400, width=400, interactive=False)
                name_display = gr.Textbox(label="이름", interactive=False)
                message_display = gr.Textbox(label="꾸며주는 말", interactive=False)
                btn_settings = gr.Button("설정")
                btn_goto_sent_letter = gr.Button("편지 보내기")
                btn_get_chat = gr.Button("상담 받기")
            
            # 설정 화면
            with gr.Column(visible=False) as setting_page:
                gr.Markdown("# 설정")
                setting_image = gr.Image(label="사진을 업로드하세요")
                setting_name = gr.Textbox(lines=1, placeholder="이름을 입력하세요", label="이름")
                setting_decorative_words = gr.Textbox(lines=1, placeholder="나에게 어떤 존재였나요", label="꾸며주는 말")
                setting_relation = gr.Radio(["어머니", "아버지", "배우자", "자녀","연인", "형제/친척", "친구","반려동물", "해당되지 않음"], label="어떤 관계였나요?")
                setting_closeness = gr.Slider(0, 10, step=1, label="얼마나 가까웠나요?", info="왼쪽에서 오른쪽으로 갈수록 가까웠던 관계에요")
                
                btn_setting_submit = gr.Button("저장 후 메인으로")

            # 편지 발신 및 답장 화면
            with gr.Column(visible=False) as sent_letter_page:
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("# 발신")
                        image_letter = gr.Image(label="고인과 함께했던 추억이 담긴 이미지를 업로드 해주세요")
                        title_letter = gr.Textbox(label="편지 제목", lines=1, placeholder="편지 제목을 작성해주세요")
                        text_letter = gr.Textbox(label="편지 내용", lines=10, placeholder="고인과 함께 했던 추억을 떠올리며 편지를 작성해주세요")

                        with gr.Row():
                            btn_goto_prevPage = gr.Button("뒤로가기")
                            btn_sent_letter = gr.Button("편지 발송하기")

                    with gr.Column():
                        gr.Markdown("# 수신")
                        reply_content = gr.Textbox(label="답장 내용", lines=25, interactive=False)

            with gr.Column(visible=False) as chat_page:
                chatbot=gr.ChatInterface(process_chat, title="마음을 털어놓아주세요", description="간단하게 최근에는 어떤 하루를 보내고 계신지 말해주세요",fill_height=False)
                btn_settings_3 = gr.Button("메인으로 넘어가기")

    # 버튼 클릭 이벤트 처리
    btn_settings_1.click(fn=lambda: switch_tab(1), inputs=None, outputs=[start_page, main_page])

    # 채팅 후 메인 페이지에 적용
    # btn_settings_1.click(user_info, inputs=user_info["name"], outputs=name_display)
    btn_settings_1.click(save_name_settings, inputs=user_info['name'], outputs=[name_display])
    btn_settings_1.click(save_text_settings, inputs=user_info['text'], outputs=[message_display])

    # 메인페이지 <-> 설정
    btn_settings.click(switch_tab, inputs=gr.State(1), outputs=[main_page, setting_page])
    btn_setting_submit.click(switch_tab, inputs=gr.State(0), outputs=[main_page, setting_page])

    # 메인페이지 <-> 설정 정보 가지고 넘어가기
    btn_settings.click(save_name_settings, inputs=user_info['name'], outputs=[setting_name])
    btn_settings.click(save_text_settings, inputs=user_info['text'], outputs=[setting_decorative_words])

    # 메인페이지 <-> 편지 발신 화면 전환
    btn_goto_sent_letter.click(switch_tab, inputs=gr.State(1), outputs=[main_page, sent_letter_page])
    btn_sent_letter.click(create_letter, inputs=[image_letter, title_letter, text_letter], outputs=reply_content)
    btn_goto_prevPage.click(switch_tab, inputs=gr.State(0), outputs=[main_page, sent_letter_page])

    # 설정 후 메인 페이지에 적용
    btn_setting_submit.click(save_settings, inputs=[setting_image, setting_name, setting_decorative_words], outputs=[image_display, name_display, message_display])
    btn_setting_submit.click(setting_gemini, inputs=[setting_relation, setting_closeness])

    # 상담 받기
    btn_get_chat.click(switch_tab, inputs=gr.State(1), outputs=[main_page, chat_page])
    btn_settings_3.click(fn=lambda: switch_tab(1), inputs=None, outputs=[chat_page, main_page])
    

demo.launch()

/opt/anaconda3/lib/python3.11/site-packages/gradio/utils.py:985: UserWarning: Expected 1 arguments for function <function save_name_settings at 0x16ae12480>, received 0.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/gradio/utils.py:989: UserWarning: Expected at least 1 arguments for function <function save_name_settings at 0x16ae12480>, received 0.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/gradio/utils.py:985: UserWarning: Expected 1 arguments for function <function save_text_settings at 0x16ae10900>, received 0.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/gradio/utils.py:989: UserWarning: Expected at least 1 arguments for function <function save_text_settings at 0x16ae10900>, received 0.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/lib/python3.11/site-packages/gradio/helpers.py:947: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
